In [1]:
import numpy as np
import pandas as pd
import random
import re
import itertools
from IPython.display import clear_output

In [ ]:
# Additional helper functions required for the code to run:
# This function defines the column headers for the data frames
def getColumnHeaders():
    return pd.Series(data=['Index','label','integer_1','integer_2','integer_3',
                                 'integer_4','integer_5','integer_6','integer_7','integer_8','integer_9',
                                 'integer_10','integer_11','integer_12','integer_13','categorical_1',
                                 'categorical_2','categorical_3','categorical_4','categorical_5','categorical_6',
                                 'categorical_7','categorical_8','categorical_9','categorical_10','categorical_11',
                                 'categorical_12','categorical_13','categorical_14','categorical_15','categorical_16',
                                 'categorical_17','categorical_18','categorical_19','categorical_20','categorical_21',
                                 'categorical_22','categorical_23','categorical_24','categorical_25','categorical_26'])

# this function generateNIndecesFrom takes a range of Indeces and randomly takes n of them, returns a pandas Series object
def generateNIndecesFrom(n, rangeOfIndeces):
    # print("Generating " + str(n) + " indeces from range")
    allIndeces = random.sample(rangeOfIndeces, n)
    allIndeces = pd.Series(data = allIndeces)
    allIndeces = allIndeces.sort_values().reset_index().drop(['index'],axis=1)
    allIndeces.columns = ['Index'];
    return allIndeces

# the generateSubSet function takes in a file, a dataFrame to put the data in as well as index values which should be extracted,
# a number of rows per itteration (this is used to not overload the memory) , total number of rows in the file, the column headers for the new dataframe
def generateSubSet(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers):
    totalNumIterations = int(totalNumRows/numRowsPerItteration)
    # print("Number of itterations = " + str(totalNumIterations))
    totalNumRowsTraversed = 0
    prevsize = 0
    for i in range(totalNumIterations + 1):
#         
#         print("Itteration number: " + str(i))
#         print("skipRows: " + str(i * numRowsPerItteration))
#         print("Read in : " + str(numRowsPerItteration))
        curData = pd.read_table(file,skiprows = i * numRowsPerItteration, nrows = numRowsPerItteration,header=None)
        curData.index = [i for i in range(i*numRowsPerItteration,i*numRowsPerItteration + curData.shape[0])]
        totalNumRowsTraversed = totalNumRowsTraversed + curData.shape[0]
        clear_output()
#         print(curData.head())
#         print(curData.shape)
#         print(curData.index.shape)

        curData['Index'] = curData.index
        curData.columns = column_headers
        
        curIndexRange = indexValues['Index'][(indexValues['Index'] < (i*numRowsPerItteration + numRowsPerItteration)) & (indexValues['Index'] > (i*numRowsPerItteration-1))]
        print(curIndexRange)
        print(curData.head())
        curData = curData[curData['Index'].isin(list(curIndexRange))] # this line isn't working for some reason
        print(curData.head())
        
        dataFrame = pd.concat([dataFrame,curData])
        
                
        
        print(dataFrame.head())
        print(curData.head())
        
        
        print("Extraction Stats: " + str(dataFrame.shape[0]) + " percent: " + str(dataFrame.shape[0] / indexValues.shape[0] * 100) + "%")
        print("Document Stats: " + str(totalNumRowsTraversed) + " percent: " + str(totalNumRowsTraversed/totalNumRows*100) + "%")
        if (dataFrame.shape[0] - prevsize) > 500000:
            prevsize = dataFrame.shape[0]
#             dataFrame.to_csv(frameSaveName)
#         elif dataFrame.shape[0] == indexValues.shape[0]:
#             print("Finished with the data collection")
# #             dataFrame.to_csv(frameSaveName)
#             break
    # print("Extraction is Done, now saving frame")        
    return dataFrame

# This method generates is a wrapper around the generateSubset to generate the subset and save the dataframe to a csv file (for being able to make use of it after)
def generateAndSaveSubset(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers,frameSaveName):
    dataFrame = generateSubSet(file,dataFrame,indexValues,numRowsPerItteration,totalNumRows,column_headers)
    dataFrame.to_csv(frameSaveName)
    return dataFrame

def read_data(data_path, train_path, validation_path, test_path):

    print(data_path)
    print(train_path)
    print(validation_path)
    print(test_path)
    
    #get the ids
    try:
        trainIndeces = pd.read_csv(train_path, header = None)
        validationIndeces = pd.read_csv(validation_path, header = None)
        testingIndeces = pd.read_csv(test_path, header = None)
    except:
        print("There were not 1000000 data points")
        trainIndeces = generateNIndecesFrom(1000000,list(twoMIndeces['Index']))
        trainIndeces.to_csv('train_ids.txt',index=False,header=False)

        remainingIndeces = twoMIndeces['Index'][~twoMIndeces['Index'].isin(trainIndeces.values)]
        validationIndeces = generateNIndecesFrom(250000,list(remainingIndeces))
        validationIndeces.to_csv('validation_ids.txt',index=False,header=False)

        testingIndeces = twoMIndeces['Index'][~(twoMIndeces['Index'].isin(trainIndeces.values) | twoMIndeces['Index'].isin(validationIndeces.values))]
        testingIndeces = generateNIndecesFrom(750000,list(testingIndeces))
        testingIndeces.to_csv('test_ids.txt',index=False,header=False)
    
    trainIndeces.columns = ['Index']
    validationIndeces.columns = ['Index']
    testingIndeces.columns = ['Index']
#     print(trainIndeces.head())
#     print(validationIndeces.head())
#     print(testingIndeces.head())
    
    # Generate the actual data files
    column_headers = getColumnHeaders()
    # print("No 1M collection")
    train1M = pd.DataFrame()
    train1M = generateSubSet(data_path,train1M,trainIndeces,4000000,46000000,column_headers)
    print(train1M.head())

    # print("No 250k collection")
    validation250k = pd.DataFrame()
    validation250k = generateSubSet(data_path,validation250k,validationIndeces,4000000,46000000,column_headers)


    # print("No 750k collection")
    test750k = pd.DataFrame()
    test750k = generateSubSet(data_path,test750k,validationIndeces,4000000,46000000,column_headers)
    
    
    print(train1M.shape)
    print(validation250k.shape)
    print(test750k.shape)

    # train_data, train_target = np.zeros((1000000, 39)), np.zeros((1000000,))
    # validation_data, validation_target = np.zeros((250000, 39)), np.zeros((250000,))
    # test_data, test_target = np.zeros((750000, 39)), np.zeros((750000,))


    return train1M[train1M.columns[1:40]], train1M['label'].values(), validation250k[validation250k.columns[1:40]], validation250k['label'].values(), test750k[test750k.columns[1:40]], test750k['label'].values()
#     return 

def preprocess_int_data(data, features):
    n = len([f for f in features if f < 13])
    return np.zeros((data.shape[0], n))


def preprocess_cat_data(data, features, preprocess):
    return None


In [ ]:
def check_output_features(fs):
    assert len(fs) < 39


def check_output_read_data(data, target, n):
    assert data.shape[0] == n
    assert target.shape[0] == n


def check_output_preprocess(preprocess):
    assert (preprocess == 'onehot') or (preprocess == 'rate') or (preprocess == 'tfidf')


def check_output_preprocess_int_data(data, fs):
    n = len([f for f in fs if f < 13])
    assert data.shape[1] == n


def check_output_preprocess_cat_data(data, fs, preprocess):
    pass


def read_features(path):
    features = []
    with open(path) as f:
        for line in f:
            features.append(int(line.strip()))
    return features


def read_preprocess(path):
    with open(path) as f:
        for line in f:
            return line.strip()


if __name__ == '__main__':
    data = './dac/train.txt'
    train = 'train_ids.txt'
    validation = 'validation_ids.txt'
    test = 'test_ids.txt'
    features = 'features.txt'
    preprocess = 'preprocess.txt'

    train_data, train_target, validation_data, validation_target, test_data, test_target = \
        read_data(data, train, validation, test)

    check_output_read_data(train_data, train_target, 1000000)
    check_output_read_data(validation_data, validation_target, 250000)
    check_output_read_data(test_data, test_target, 750000)

    features = read_features(options.features)

    check_output_features(features)

    preprocess = read_preprocess(options.preprocess)

    check_output_preprocess(prepr3ocess)

    train_int_data = preprocess_int_data(train_data, features)
    validation_int_data = preprocess_int_data(validation_data, features)
    test_int_data = preprocess_int_data(test_data, features)

    check_output_preprocess_int_data(train_int_data, features)
    check_output_preprocess_int_data(validation_int_data, features)
    check_output_preprocess_int_data(test_int_data, features)

    train_cat_data = preprocess_cat_data(train_data, features, preprocess)
    validation_cat_data = preprocess_cat_data(validation_data, features, preprocess)
    test_cat_data = preprocess_cat_data(test_data, features, preprocess)

    check_output_preprocess_cat_data(train_cat_data, features, preprocess)
    check_output_preprocess_cat_data(validation_cat_data, features, preprocess)
    check_output_preprocess_cat_data(test_cat_data, features, preprocess)


87443     4000075
87444     4000109
87445     4000172
87446     4000199
87447     4000251
87448     4000262
87449     4000366
87450     4000372
87451     4000388
87452     4000389
87453     4000466
87454     4000554
87455     4000571
87456     4000619
87457     4000628
87458     4000651
87459     4000660
87460     4000711
87461     4000712
87462     4000731
87463     4000789
87464     4000790
87465     4000852
87466     4000858
87467     4001060
87468     4001088
87469     4001118
87470     4001157
87471     4001170
87472     4001199
           ...   
174698    7999009
174699    7999032
174700    7999040
174701    7999059
174702    7999065
174703    7999081
174704    7999105
174705    7999139
174706    7999171
174707    7999183
174708    7999213
174709    7999275
174710    7999277
174711    7999281
174712    7999309
174713    7999343
174714    7999370
174715    7999420
174716    7999430
174717    7999462
174718    7999542
174719    7999566
174720    7999601
174721    7999650
174722    